In [3]:
from tifffile import TiffFile
import matplotlib.pyplot as plt
from matplotlib import colors
import numpy as np
import os
import time as t
import tensorflow as tf
from tensorflow import keras
import pickle

construction de x_train

In [4]:
# meme nom dans images et masks
files_train = os.listdir("./dataset_EI4/train/images")

x_train_liste = []

for i in files_train:
    with TiffFile('./dataset_EI4/train/images/' + i) as tif:
       x_train_liste.append(tif.asarray())


x_train = np.array(x_train_liste)

128.6632318496704


construction de y_train

In [5]:
# meme nom dans images et masks
files_train = os.listdir("./dataset_EI4/train/images")

y_train_liste = []

for i in files_train:

    with TiffFile('./dataset_EI4/train/masks/' + i) as tif:
        y_train_liste.append(tif.asarray())


y_train = np.asarray(y_train_liste)

40.32776737213135


construction de x_test

In [6]:
# meme nom dans images et masks
files_train = os.listdir("./dataset_EI4/test/images")

x_test_liste = []

for i in files_train:
    with TiffFile('./dataset_EI4/test/images/' + i) as tif:
       x_test_liste.append(tif.asarray())

x_test = np.array(x_test_liste)

22.881664752960205


construction de y_test

In [7]:
# meme nom dans images et masks
files_train = os.listdir("./dataset_EI4/test/images")

y_test_liste = []

for i in files_train:
    with TiffFile('./dataset_EI4/test/masks/' + i) as tif:
       y_test_liste.append(tif.asarray())
       
y_test = np.array(y_test_liste)

12.299220323562622


In [8]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(18491, 256, 256, 4)
(18491, 256, 256)
(5043, 256, 256, 4)
(5043, 256, 256)


In [11]:
#normalisation dans [0,1] reduit la dispersion
#x_train /= 255
#x_test /= 255



UFuncTypeError: Cannot cast ufunc 'divide' output from dtype('float64') to dtype('uint16') with casting rule 'same_kind'

# Test de classification segmentée

In [9]:
class_names = ['nodata', 'clouds', 'artificial surfaces and construction​','cultivated areas​','broadleaf tree cover​','coniferous tree cover​','herbaceous vegetation​','natural material surfaces​','permanent snow-covered surfaces​','water bodies​']

dataset = tf.data.Dataset({'train':})

In [8]:
TRAIN_LENGTH = 18491
BATCH_SIZE = 50
BUFFER_SIZE = 1000
STEPS_PER_EPOCH = TRAIN_LENGTH // BATCH_SIZE

## Définir le réseau de neurones

Pour définir chaque couche de notre réseau de neurones, on va d'abord :
- initialiser le modèle avec : `model=tf.keras.models.Sequential()` (c'est un modèle séquentiel i.e. où les couches vont s'enchaîner)
- ajouter des couches avec : `model.add(couche)`

Les couches qu'on va ajouter :
- d'abord une couche `Flatten(input_shape=(im_height,im_width,n_channels))` qui va servir à aplanir l'image en un vecteur. (n_channels= 3 si image en couleur, = 1 si image en noir et blanc)
 `input_shape=` est un argument obligatoire pour la 1ère couche du modèle, cela détermine la taille des données. Pour les couches suivantes ce n'est plus utile car la taille intermédiaire est calculée automatiquement.
 - 2 couches `Dense(nb_neurones,activation='à vous de choisir')`, ce sont les couches de neurones intermédiaires (pas besoin de refaire tous les produits matriciels horribles OMG)
 - une couche de sortie `Dense(n_classes,activation='softmax')`, on prédit un vecteur probabilités de taille `n_classes`. La fonction d'activation softmax permet d'avoir un vecteur de probabilité en sortie.

 Exemple:
 ```
  model.add(keras.layers.Dense(64,activation='relu'))
 ```
 Documentation sur comment faire un modèle séquentiel : https://www.tensorflow.org/guide/keras/sequential_model

In [40]:
### Define the neural network
from tensorflow import keras
from tensorflow.keras import layers

'''def neural_network():
    model=tf.keras.models.Sequential()
    model.add(keras.layers.Flatten(input_shape=(256,256,4)))
    model.add(keras.layers.Dense(512,activation='relu'))
    model.add(keras.layers.Dense(256,activation='relu'))
    model.add(keras.layers.Dense(10,activation='softmax'))

    return model'''

model = keras.Sequential([
    keras.Input(shape=(256, 256, 4)),
    layers.Reshape(target_shape=(256, 256,4)),
    layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dropout(0.5),
    layers.Dense(256, activation="softmax")
])

In [41]:
#creation du modele
model = neural_network()

In [42]:
# Visualisez votre réseau de neurones
model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_12 (Flatten)        (None, 262144)            0         
                                                                 
 dense_20 (Dense)            (None, 512)               134218240 
                                                                 
 dense_21 (Dense)            (None, 256)               131328    
                                                                 
 dense_22 (Dense)            (None, 10)                2570      
                                                                 
Total params: 134,352,138
Trainable params: 134,352,138
Non-trainable params: 0
_________________________________________________________________


In [36]:
### Define the optimizer,loss function and metrics
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
loss = tf.keras.losses.SparseCategoricalCrossentropy()
metrics = ['accuracy']

In [37]:
#compilation du modèle
#model.compile(opt,loss,metrics)
#gpt :
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Maintenant vous allez entrainer votre modèle sur le dataset qui comporte les images (x_train) associées à leur label (y_train). Pour cela on utilise la méthode `model.fit(...)`.

L'argument epochs correspond au nombre de fois où le dataset est présenté au réseau de neurones. Ici mettez entre 1 et 20 epochs.

L'argument validation_data sert à vérifier que votre modèle est bon sur des images qu'il n'a jamais vues (pas utilisées dans l'entrainement), i.e. qu'il n'a pas "appris" le dataset par cœur.

Documentation : https://www.tensorflow.org/api_docs/python/tf/keras/Model

In [39]:
history = model.fit(x_train,y_train,epochs=2,validation_data=(x_test,y_test),batch_size=40)

Epoch 1/2


ValueError: in user code:

    File "c:\Users\clemm\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\clemm\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\clemm\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\clemm\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1024, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "c:\Users\clemm\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1082, in compute_loss
        return self.compiled_loss(
    File "c:\Users\clemm\